# Imports

In [1]:
import os
import sys

import numpy as np
from tqdm import tqdm
import cv2

from src.sam_segmentation.yolo import load_yolov8_detector, yolov8_detect
from src.sam_segmentation.sam import load_sam_predictor, sam_segmentation
from src.sam_segmentation.utils import unite_masks, masks_narrowing

# Data

In [4]:
DATA_DIR = r"C:\Internship\ITMO_ML\data\raw_frames\Frames_bubbles\Bubbles_every_frame"

In [5]:
folders_list = os.listdir(DATA_DIR)
folders_list

['F1_1_1_1_frames',
 'F1_1_1_2_frames',
 'F1_1_2_1_frames',
 'F1_1_2_2_frames',
 'F1_1_3_1_frames',
 'F1_1_3_2_frames',
 'F1_1_4_1_frames',
 'F1_1_4_2_frames',
 'F1_1_5_1_frames',
 'F1_1_5_2_frames',
 'F1_2_2_1_frames',
 'F1_2_2_2_frames',
 'F1_2_3_1_frames',
 'F1_2_3_2_frames']

In [6]:
def count_images(image_folder_path):
    images_num = len(os.listdir(image_folder_path))
    return images_num

def summarize_folders(data_dir, folders_list):
    folders_info = {}
    images_sum = 0
    
    for folder in folders_list:
        images_num = count_images(os.path.join(data_dir, folder))
        folders_info[folder] = images_num
        images_sum += images_num
        
    print(f"There is {images_sum} images:")
    for folder in folders_info.keys():
        print(f"\t{folder} folder has {folders_info[folder]} images")
    

In [7]:
summarize_folders(DATA_DIR, folders_list)

There is 20721 images:
	F1_1_1_1_frames folder has 1787 images
	F1_1_1_2_frames folder has 1798 images
	F1_1_2_1_frames folder has 1508 images
	F1_1_2_2_frames folder has 1491 images
	F1_1_3_1_frames folder has 1479 images
	F1_1_3_2_frames folder has 1512 images
	F1_1_4_1_frames folder has 1797 images
	F1_1_4_2_frames folder has 1783 images
	F1_1_5_1_frames folder has 1502 images
	F1_1_5_2_frames folder has 1493 images
	F1_2_2_1_frames folder has 1474 images
	F1_2_2_2_frames folder has 1484 images
	F1_2_3_1_frames folder has 1480 images
	F1_2_3_2_frames folder has 133 images


# SAM+YOLO Labeling

Yolo detector:

In [8]:
custom_yolo_checkpoint_path = r"C:\Internship\ITMO_ML\CTCI\checkpoints\yolov8\yolov8s_on_bubbles_24\run3\weights\best.pt"
detector = load_yolov8_detector(custom_yolo_checkpoint_path)

SAM:

In [9]:
sam_checkpoint = r"C:\Internship\ITMO_ML\CTCI\checkpoints\sam_checkpoints\sam_vit_h_4b8939.pth"
model_type = "vit_h"
device="cuda"

predictor = load_sam_predictor(checkpoint_path=sam_checkpoint, model_type=model_type, device=device)

## Labeling

In [10]:
def segment_images_from_folder(folder, narrowing=0.20):
    masks_folder_name = folder + "_masks"
    dirlist = os.listdir(DATA_DIR)
    if masks_folder_name not in dirlist:
        os.mkdir(os.path.join(DATA_DIR, masks_folder_name))
    
    images_list = os.listdir(os.path.join(DATA_DIR, folder))
    masks_list = os.listdir(os.path.join(DATA_DIR, masks_folder_name))
    for image_name in tqdm(images_list):
        if image_name in masks_list:
            continue
        image = cv2.imread(os.path.join(DATA_DIR, folder,image_name))
        boxes = yolov8_detect(image=image, detector=detector,    return_objects=False)
        masks_list = sam_segmentation(image=image,predictor=predictor,  boxes=boxes, prompt_point=True)
    
        masks_united = []
        for masks in masks_list:
            masks_united.append(unite_masks(masks))
        masks_narrowed = masks_narrowing(masks_united, narrowing=narrowing)
        mask = unite_masks(masks_narrowed)
        cv2.imwrite(filename=os.path.join(DATA_DIR,masks_folder_name,   image_name), img=mask)

    print(f"Images from the folder {folder} were segmented!")

In [11]:
folder = folders_list[1]
narrowing = 0.25

In [ ]:
segment_images_from_folder(folder, narrowing=narrowing)

  0%|          | 0/1798 [00:00<?, ?it/s]

0: 608x800 300 Bubbless, 145.6ms
Speed: 3.0ms preprocess, 145.6ms inference, 52.9ms postprocess per image at shape (1, 3, 608, 800)
